# Data Scientist Professional Practical Exam Submission

**Use this template to write up your summary for submission. Code in Python or R needs to be included.**


## 📝 Task List

Your written report should include both code, output and written text summaries of the following:
- Data Validation:   
  - Describe validation and cleaning steps for every column in the data 
- Exploratory Analysis:  
  - Include two different graphics showing single variables only to demonstrate the characteristics of data  
  - Include at least one graphic showing two or more variables to represent the relationship between features
  - Describe your findings
- Model Development
  - Include your reasons for selecting the models you use as well as a statement of the problem type
  - Code to fit the baseline and comparison models
- Model Evaluation
  - Describe the performance of the two models based on an appropriate metric
- Business Metrics
  - Define a way to compare your model performance to the business
  - Describe how your models perform using this approach
- Final summary including recommendations that the business should undertake

*Start writing report here..*

## 0. Library and data import

In [2]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('recipe_site_traffic_2212.csv')
print(df.shape)
df.head(10)

(947, 8)


,recipe,calories,carbohydrate,sugar,protein,category,servings,high_traffic
0,1,NaN,NaN,NaN,NaN,Pork,6,High
1,2,35.48,38.56,0.66,0.92,Potato,4,High
2,3,914.28,42.68,3.09,2.88,Breakfast,1,NaN
3,4,97.03,30.56,38.63,0.02,Beverages,4,High
4,5,27.05,1.85,0.80,0.53,Beverages,4,NaN
5,6,691.15,3.46,1.65,53.93,One Dish Meal,2,High
6,7,183.94,47.95,9.75,46.71,Chicken Breast,4,NaN
7,8,299.14,3.17,0.40,32.40,Lunch/Snacks,4,NaN
8,9,538.52,3.78,3.37,3.79,Pork,6,High
9,10,248.28,48.54,3.99,113.85,Chicken,2,NaN


## 1. Data Exploration, Validation and Cleaning

In [4]:
# Let's see some basic info, like the number of rows, columns and data type of columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947 entries, 0 to 946
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   recipe        947 non-null    int64  
 1   calories      895 non-null    float64
 2   carbohydrate  895 non-null    float64
 3   sugar         895 non-null    float64
 4   protein       895 non-null    float64
 5   category      947 non-null    object 
 6   servings      947 non-null    object 
 7   high_traffic  574 non-null    object 
dtypes: float64(4), int64(1), object(3)
memory usage: 59.3+ KB


The 'calories,' 'carbohydrate,' 'sugar,' and 'protein' columns remained unchanged, with a 'numeric' data type. However, the 'category,' 'servings,' and 'high_traffic' columns were converted to the 'categorical' data type


In [5]:
df['category'] = df['category'].astype('category')
df['high_traffic'] = df['high_traffic'].astype('category')

We'll rename 'high_traffic' column to 'Traffic', leave values of 'High' for instances of high trafficand replaced the empty values with 'Low'.

In [6]:
# Fixing the high_traffic column
print(df['high_traffic'].value_counts())
df['high_traffic'] = df['high_traffic'].cat.add_categories('Low')
df["high_traffic"] = df["high_traffic"].fillna('Low')
df.rename(columns={'high_traffic': 'traffic'}, inplace=True)
print(df['traffic'].value_counts())

High    574
Name: high_traffic, dtype: int64
High    574
Low     373
Name: traffic, dtype: int64


There are 947 rows and 8 columns. 

Column servings is strangly of Dtype object, while it should be integer or float.

We found that a few alphabetical characters slipped into the column, but they can easily be replaced and converted to numeric.

In [7]:
df.servings.value_counts()

4               389
6               197
2               183
1               175
4 as a snack      2
6 as a snack      1
Name: servings, dtype: int64

In [8]:
#cleaning servings
df.loc[df.servings == '4 as a snack', 'servings'] = 4

df.loc[df.servings == '6 as a snack', 'servings'] = 6

df['servings'] = df['servings'].astype('int')

#checking our results
df.servings.value_counts()

4    391
6    198
2    183
1    175
Name: servings, dtype: int64

In [9]:
df[df.servings.isnull()]

,recipe,calories,carbohydrate,sugar,protein,category,servings,traffic


The category column is supposed to contain 10 specified categories: 'Lunch/Snacks', 'Beverages', 'Potato', 'Vegetable', 'Meat', 'Chicken', 'Pork', 'Dessert', 'Breakfast', 'One Dish Meal'. There was one additional category "Chicken breast" which can be reassigned to 'Chicken' category.

In [10]:
df.category.value_counts()

Breakfast         106
Chicken Breast     98
Beverages          92
Lunch/Snacks       89
Potato             88
Pork               84
Dessert            83
Vegetable          83
Meat               79
Chicken            74
One Dish Meal      71
Name: category, dtype: int64

In [11]:
# Standardizing the category for chicken
df['category'] = df['category'].replace('Chicken Breast', 'Chicken')

# Checking the updated category counts
df['category'].value_counts()

Chicken          172
Breakfast        106
Beverages         92
Lunch/Snacks      89
Potato            88
Pork              84
Dessert           83
Vegetable         83
Meat              79
One Dish Meal     71
Name: category, dtype: int64

In [12]:
# Let's see the statistics of the numerical columns
df.describe()

,recipe,calories,carbohydrate,sugar,protein,servings
count,947.000000,895.000000,895.000000,895.000000,895.000000,947.000000
mean,474.000000,435.939196,35.069676,9.046547,24.149296,3.477297
std,273.519652,453.020997,43.949032,14.679176,36.369739,1.732741
min,1.000000,0.140000,0.030000,0.010000,0.000000,1.000000
25%,237.500000,110.430000,8.375000,1.690000,3.195000,2.000000
50%,474.000000,288.550000,21.480000,4.550000,10.800000,4.000000
75%,710.500000,597.650000,44.965000,9.800000,30.200000,4.000000
max,947.000000,3633.160000,530.420000,148.750000,363.360000,6.000000


In [13]:
# Let's check the amount of missing values per column
df.isna().sum()

recipe           0
calories        52
carbohydrate    52
sugar           52
protein         52
category         0
servings         0
traffic          0
dtype: int64

Nutritional columns (calories, carbohydrate, sugar and protein), have 52 missings values. Seems that they are missing in pairs, all together per recipe. 

This represents more than 5% of data, so we must clean it, rathar than dropping values.

In [14]:
# Let's first ensure that there are no duplicate values, ie. 'recipe' values are unique (verification step)
df['recipe'].is_unique

True

In [15]:
# Drop duplicates
df.drop_duplicates(inplace=True)
df.shape

(947, 8)

We see that there are no duplicate values.

**1.2 Dealing with missing values in nutritional columns**

Most common way to replace missing numerical values is to assing the median value of the column. But as the data has a mixture of values for different categories of food, as well as, different amount of servings, we should maybe use median value per category and serving. 

But maybe this is not as important for the task of this study, which is to predict recipes that will have a high traffic. So we must first explore the data with EDA.

**Update after first training** We'll fill NaN values in the nutrients from madian values per category and check if the model performs better.

In [16]:
# Calculate calories per serving
df['calories_per_serving'] = df['calories'] / df['servings']
df['protein_per_serving'] = df['protein'] / df['servings']
df['sugar_per_serving'] = df['sugar'] / df['servings']
df['carbohydrate_per_serving'] = df['carbohydrate'] / df['servings']

In [19]:
# Fill NaN values in 'calories', 'protein', 'sugar', 'carbohydrate' columns with the mean value for each per serving category
df['calories_per_serving'] = df.groupby(['category','servings'])[['calories_per_serving']].transform(lambda x: x.fillna(x.mean()))
df['protein_per_serving'] = df.groupby(['category','servings'])['protein_per_serving'].transform(lambda x: x.fillna(x.mean()))
df['sugar_per_serving'] = df.groupby(['category','servings'])['sugar_per_serving'].transform(lambda x: x.fillna(x.mean()))
df['carbohydrate_per_serving'] = df.groupby(['category','servings'])['carbohydrate_per_serving'].transform(lambda x: x.fillna(x.mean()))

In [20]:
# Last check - Let's check the amount of missing values per column
df.isna().sum()

recipe                       0
calories                    52
carbohydrate                52
sugar                       52
protein                     52
category                     0
servings                     0
traffic                      0
calories_per_serving         0
protein_per_serving          0
sugar_per_serving            0
carbohydrate_per_serving     0
dtype: int64

We see now that we filled missing values, but only for feature engineered columns. Let's check those that are missings for calories, carbs, sugar or protein.

In [21]:
# Let's check those values that are missing
df[df.sugar.isnull()]

,recipe,calories,carbohydrate,sugar,protein,category,servings,traffic,calories_per_serving,protein_per_serving,sugar_per_serving,carbohydrate_per_serving
0,1,NaN,NaN,NaN,NaN,Pork,6,High,127.440222,11.088000,1.168889,3.345333
23,24,NaN,NaN,NaN,NaN,Meat,2,Low,318.206538,19.829615,4.612308,11.011154
48,49,NaN,NaN,NaN,NaN,Chicken,4,Low,142.789094,9.172862,1.363442,6.321014
82,83,NaN,NaN,NaN,NaN,Meat,4,High,142.808897,10.219191,1.360662,5.007059
89,90,NaN,NaN,NaN,NaN,Pork,6,High,127.440222,11.088000,1.168889,3.345333
116,117,NaN,NaN,NaN,NaN,Chicken,6,High,96.448056,10.152056,0.824556,3.172056
121,122,NaN,NaN,NaN,NaN,Dessert,2,High,128.080385,2.643077,18.045769,21.404615
136,137,NaN,NaN,NaN,NaN,One Dish Meal,2,High,267.183409,17.568864,2.793182,14.939545
149,150,NaN,NaN,NaN,NaN,Potato,2,High,156.101500,2.928500,1.485000,34.764000
187,188,NaN,NaN,NaN,NaN,Pork,4,High,143.964621,10.055455,2.198939,7.272045


In [22]:
# Let's try to find some ilogical situation, ei. sugar content is higher than carbs
df[df.sugar > df.carbohydrate]

,recipe,calories,carbohydrate,sugar,protein,category,servings,traffic,calories_per_serving,protein_per_serving,sugar_per_serving,carbohydrate_per_serving
3,4,97.03,30.56,38.63,0.02,Beverages,4,High,24.257500,0.0050,9.6575,7.640000
11,12,155.80,8.27,9.78,11.55,Breakfast,6,Low,25.966667,1.9250,1.6300,1.378333
14,15,217.14,6.69,10.00,15.17,Meat,4,High,54.285000,3.7925,2.5000,1.672500
15,16,316.45,2.65,4.68,79.71,Meat,6,High,52.741667,13.2850,0.7800,0.441667
16,17,454.27,1.87,2.95,61.07,Meat,2,High,227.135000,30.5350,1.4750,0.935000
...,...,...,...,...,...,...,...,...,...,...,...,...
916,917,64.27,0.16,51.64,0.74,Beverages,2,Low,32.135000,0.3700,25.8200,0.080000
924,925,199.20,0.69,4.11,0.12,Beverages,4,Low,49.800000,0.0300,1.0275,0.172500
935,936,262.29,60.13,72.49,5.07,Dessert,4,High,65.572500,1.2675,18.1225,15.032500
936,937,164.86,1.62,7.57,48.35,Pork,4,High,41.215000,12.0875,1.8925,0.405000


## ✅ When you have finished...
-  Publish your Workspace using the option on the left
-  Check the published version of your report:
	-  Can you see everything you want us to grade?
    -  Are all the graphics visible?
-  Review the grading rubric. Have you included everything that will be graded?
-  Head back to the [Certification Dashboard](https://app.datacamp.com/certification) to submit your practical exam report and record your presentation